In [1]:
region_name = "TDS" # "MDIS_10-800" # "TDS" # _ELAISN1
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+"_cat.fits"

srcs_ids = []
srcs_ids = [1891 ,3403 ,16149 ,45461 ,] # WDs TDS


In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import unique, Table, hstack, join, setdiff
from astropy import units as uu
from astroquery.simbad import Simbad
from astropy.visualization import quantity_support
from astropy.modeling.models import BlackBody
from astropy import constants as cc
from matplotlib.ticker import ScalarFormatter
from astroquery.sdss import SDSS

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis
from vasca.tables import TableCollection, dd_vasca_columns
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, dd_filter2lambda, mag2flux, flux2mag

rg = Region()
rg.load_from_fits(region_fname)
if len(srcs_ids)>0:
    rg.tt_sources.add_index("rg_src_id")
    idx_srcs = rg.tt_sources.loc_indices["rg_src_id", srcs_ids]
    tt_srcs = Table(rg.tt_sources[idx_srcs])
elif only_sel_srcs:
    tt_srcs = rg.tt_sources[rg.tt_sources["sel"]]
else:
    tt_srcs = rg.tt_sources

nr_srcs = len(tt_srcs)
tt_srcs.add_index("rg_src_id")
display(tt_srcs)

fd_src_id,nr_det,ra,dec,pos_err,pos_xv,pos_var,pos_cpval,pos_rchiq,coadd_src_id,coadd_dist,obs_filter_id,sel,flux,flux_err,flux_nxv,flux_var,flux_cpval,flux_rchiq,coadd_ffactor,coadd_fdiff_s2n,rg_fd_id,rg_src_id,nr_fd_srcs,hr,hr_err,otype,main_id,otypes,z_value,distance_distance,distance_unit,distance_result
,,deg,deg,arcsec,arcsec2,arcsec2,,,,arcsec,,,1e-06 Jy,1e-06 Jy,,1e-12 Jy2,,,,,,,,,,,,,,,,arcsec
int32,int32[2],float64,float64,float32,float32,float32,float32,float32,int64,float32,int32[2],bool,float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],int64,int32,int32,float32,float32,bytes32,bytes32,bytes32,float64,float64,bytes4,float32
-1,35 .. 5,334.61900740472345,-0.0033813554025151345,0.08654309,-0.13358614,0.16820572,0.97304916,0.5630017,2410,0.18017368,1 .. 2,True,121.59981 .. 40.61505,0.32700354 .. 0.6793138,0.0012762442 .. 0.0012225002,22.828146 .. 9.024125,2.1874502e-27 .. 0.001186736,6.2001452 .. 4.5217357,1.0003184 .. 1.007664,0.08853569 .. 0.31262133,-1,1891,2,0.33295098,0.0059931264,WD*,PB 5130,WD*|WD*|WD*|WD*|*|Opt|UV|WD?|WD?,1.3299500,121.7450,pc,0.3261
-1,35 .. 4,333.5413675869932,0.8794347888579439,0.10153956,-0.2568612,0.15229508,0.99949604,0.37336546,6503,0.08153211,1 .. 2,True,20.24083 .. 5.189896,0.16559538 .. 0.40037686,0.06014167 .. -0.020882767,25.663895 .. 0.08260181,0.0 .. 0.9400651,26.422163 .. 0.13361952,0.9598218 .. 1.0394988,-3.864632 .. 0.3762283,-1,3403,2,0.34424755,0.02826839,WD*,GALEX J221409.8+005245,WD*|WD*|WD*|*|Opt|UV,--,439.0000,pc,0.2825
-1,28 .. 9,150.65361426058803,2.643111670533707,0.08557914,-0.08412055,0.19236396,0.8580123,0.66507405,44620,0.12717035,1 .. 2,True,106.96241 .. 66.50696,0.34322292 .. 0.6652977,0.0007799396 .. 0.0010192677,12.584814 .. 8.828741,5.945311e-11 .. 0.026203794,3.846406 .. 2.1749918,1.0030466 .. 1.0014558,0.71483445 .. 0.09157375,-1,16149,2,0.6084019,0.006708973,WD?,GALEX J100236.8+023834,*|*|G|Opt|UV|WD?|WD?,1.9900000,293.2980,pc,0.4992
-1,12 .. -1,244.14364033210146,54.16991388586225,0.16000032,-0.1349428,0.17470829,0.4975613,0.578193,12469,0.3236669,1 .. 0,True,46.202312 .. -1.0,0.4356251 .. -1.0,0.019377798 .. -100.0,43.763363 .. -1.0,7.33e-43 .. -1.0,20.80549 .. -1.0,1.0119411 .. -100.0,1.0512818 .. -10000.0,-1,45461,1,-1.0,-1.0,WD?,GALEX J161634.4+541011,*|*|Opt|UV|WD?|WD?,--,93.6920,pc,0.9987


In [3]:
sdss_ftr     = ["u" , "g" , "r"  ,"i" , "z"]
sdss_wav  = [3546*uu.AA, 4670*uu.AA, 6156*uu.AA ,7472*uu.AA, 8917*uu.AA]


#xid = SDSS.query_region(pos, radius='5 arcsec', spectro=False)
#ll_sp = SDSS.get_spectra(matches=xid)

def AA2ev_np(wave):
    return (cc.h*cc.c/(wave*uu.AA)).to(uu.eV).value
def ev2AA_np(ener):
    return (cc.h*cc.c/(ener*uu.eV)).to(uu.AA).value
def flux2mag_np(flux):
    return flux2mag(flux).data

def mag2flux_np(mag):
    return mag2flux(mag).data

def plot_sed(rg_src_id):
    # get SED pints (for noa assume sdss)
    tc_src = rg.get_src_from_id(rg_src_id)
    pos = SkyCoord(tc_src.tt_simbad["ra"].quantity,tc_src.tt_simbad["dec"].quantity,frame='icrs')
    tt_sdss = SDSS.query_crossid(pos, radius=0.5*uu.arcsec)
    
    ll_flux = []
    ll_flux_err = []
    for flt in sdss_ftr :
        flux,flux_err = mag2flux(tt_sdss["psfMag_"+flt],tt_sdss["psfMagerr_"+flt])
        ll_flux.append(flux[0])
        ll_flux_err.append(flux_err[0])

    tt_sed = Table()
    tt_sed["flux"] = ll_flux
    tt_sed["flux_err"] = ll_flux_err
    tt_sed["wav"] = sdss_wav

    flts = ["NUV","FUV"]
    for ii in range(len(flts)):
        if tc_src.tt_sources["flux"].quantity[:,ii][0]>0:
            tt_sed.add_row([tc_src.tt_sources["flux"].quantity[:,ii][0],tc_src.tt_sources["flux_err"].quantity[:,ii][0],dd_filter2lambda[flts[ii]]])
    tt_sed.sort("wav")
    #display(tt_sed)

    #Get spectrum
    #sdss_id = tt_sdss["objID"] [0]
    tt_xid = SDSS.query_region(pos, radius='1 arcsec', spectro=True)
    if type(tt_xid) != type(None):
        ll_sp = SDSS.get_spectra(matches=tt_xid)
        print("Nr of spectra found",len(ll_sp))

        hdu_spec = ll_sp[0]
        tt_spec = Table(hdu_spec["COADD"].data)
        spec_lam = np.power(10,tt_spec["loglam"]) * uu.AA
        spec_flux = tt_spec["flux"] * 1e-17 * uu.erg/(uu.cm**2 * uu.s *uu.AA)
        s2n = tt_spec["flux"]*np.sqrt(tt_spec["ivar"])
        sel = s2n>1
        c_Aps = cc.c.to(uu.AA/uu.s)
        sed_flux = (spec_flux* spec_lam ** 2 / c_Aps).to(uu.Jy)
    else:
        print("No spectrum found")
    
    with quantity_support():
        fig_src = plt.figure("SED",figsize=(8, 7), clear=True)
        plt.xscale('log')
        plt.yscale('log')
        plt.errorbar(tt_sed["wav"].quantity,tt_sed["flux"].quantity,yerr= tt_sed["flux_err"].quantity , fmt="o")
        if type(tt_xid) != type(None):
            plt.plot(spec_lam[sel],sed_flux[sel])
    
        ax = plt.gca()
        secax = ax.secondary_xaxis("top", functions=(AA2ev_np, ev2AA_np))
        secax.set_xlabel("eV")
    
        secay = ax.secondary_yaxis("right", functions=(flux2mag_np, mag2flux_np))
    
        # Avoid scientific notation for magnitudes
        formatter = ScalarFormatter()
        formatter.set_scientific(False)
        secay.yaxis.set_minor_formatter(formatter)
    
        secay.set_ylabel("AB magnitude")
        plt.show()

    fig_lc_src = plt.figure("Light curve",figsize=(8, 4), clear=True)
    plt.cla()
    vvis.plot_light_curve(rg, rg_src_ids = rg_src_id,fig=fig_lc_src) #
    plt.show()

    display(tc_src.tt_simbad)
    display(tt_sdss)

#bb = BlackBody(temperature=26000*uu.K, scale=2.7e-25)
#bb_lambda = np.power(10,np.arange(3, 4.2,0.01)) * uu.AA
#bb_flux = bb(bb_lambda).to(uu.Jy/uu.sr)*1*uu.sr
   
widgets.interactive(plot_sed, rg_src_id=tt_srcs["rg_src_id"])

interactive(children=(Dropdown(description='rg_src_id', options=(1891, 3403, 16149, 45461), value=1891), Outpu…